<a href="https://colab.research.google.com/github/Andrea-Giuseppe-Cavallaro/data-analysis/blob/main/airBnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importo librerie

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import plotly.graph_objects as go
import plotly.colors as pc
from google.colab import drive
import os

## Caricamento CSV e Macroanalisi

In [ ]:
# Montaggio Google Drive
drive.mount('/content/drive')
# Cambio directory di lavoro
os.chdir('/content/drive/MyDrive/Colab Notebooks/EsameDataAnalisi')

In [ ]:
# Caricamento dataset
listings = pd.read_csv('listings.csv', sep=',')
reviews = pd.read_csv('reviews.csv', sep=',')
listings_df = pd.DataFrame(listings)
reviews_df = pd.DataFrame(reviews)

In [ ]:
print(f"Dataset caricati:")
print(f"- Listings: {listings.shape[0]} righe, {listings.shape[1]} colonne")
print(f"- Reviews: {reviews.shape[0]} righe, {reviews.shape[1]} colonne")

In [ ]:
print(f"- Listings: {listings.info()}")
print(f"- Reviews: {reviews.info()}")

In [ ]:
print(f"Dataset head:")
print(f"- Listings: {listings.head()}")
print(f"- Reviews: {reviews.head()}")

In [ ]:
print(f"Dataset describe:")
print(f"- Listings: {listings.describe()}")
print(f"- Reviews: {reviews.describe()}")

In [ ]:
print(f"Dataset value_counts:")
print(f"- Listings: {listings['price'].value_counts(dropna=False)}")

In [ ]:
print(f"Dataset value_counts:")
print(f"- Reviews: {listings['room_type'].value_counts(dropna=False)}")

In [ ]:
print(f"Dataset value_counts:")
print(f"- Reviews: {listings['accommodates'].value_counts(dropna=False)}")

## Pulizia dati e costruzione dataset

In [ ]:
# Pulizia colonna price - rimozione valori null e conversione a float
listings.dropna(subset=['price'], inplace=True)
listings['price_clean'] = listings['price'].replace('[\\$,]', '', regex=True).astype(float)

In [ ]:
# Aggiunta conteggio recensioni per ogni listing
review_count = reviews['listing_id'].value_counts().reset_index()
review_count.columns = ['id', 'review_count']
listings = pd.merge(listings, review_count, on='id', how='left')

In [ ]:
# Definizione aree della provincia di Catania
catania_areas = [
    "Aci Bonaccorsi", "Aci Castello", "Aci Catena", "Aci Sant'Antonio",
    "Acireale", "Adrano", "Belpasso", "Biancavilla", "Bronte",
    "Calatabiano", "Caltagirone", "Camporotondo Etneo", "Castel di Iudica",
    "Castiglione di Sicilia", "Catania", "Fiumefreddo di Sicilia",
    "Giarre", "Grammichele", "Gravina di Catania", "Licodia Eubea",
    "Linguaglossa", "Maletto", "Maniace", "Mascali", "Mascalucia",
    "Mazzarrone", "Militello in Val di Catania", "Milo", "Mineo",
    "Mirabella Imbaccari", "Misterbianco", "Motta Sant'Anastasia",
    "Nicolosi", "Palagonia", "Paternò", "Pedara", "Piedimonte Etneo",
    "Raddusa", "Ragalna", "Ramacca", "Randazzo", "Riposto",
    "San Cono", "San Giovanni la Punta", "San Gregorio di Catania",
    "San Michele di Ganzaria", "San Pietro Clarenza",
    "Sant'Agata li Battiati", "Sant'Alfio", "Santa Maria di Licodia",
    "Santa Venerina", "Scordia", "Trecastagni", "Tremestieri Etneo",
    "Valverde", "Viagrande", "Vizzini", "Zafferana Etnea"
]

# Filtro per aree di Catania
listings = listings[listings['neighbourhood_cleansed'].isin(catania_areas)]

In [ ]:
# Conversione date e filtro per host attivi
reviews['date'] = pd.to_datetime(reviews['date'])

# Trova ultima recensione per ogni listing per identificare host attivi
latest_review_date = reviews.groupby('listing_id')['date'].max().reset_index()
latest_review_date.columns = ['id', 'latest_review_date']
listings = pd.merge(listings, latest_review_date, on='id', how='left')

# Rimuovi host con ultima recensione prima del 2024 (inattivi)
hosts_to_remove = listings[listings['latest_review_date'] < '2024-01-01']['host_id'].unique()
listings = listings[~listings['host_id'].isin(hosts_to_remove)]

# Trova prima recensione per ogni host per escludere nuovi host 2025
first_review_year = reviews.groupby('listing_id')['date'].min().dt.year.reset_index()
first_review_year.columns = ['host_id', 'first_review_year']
listings = pd.merge(listings, first_review_year, on='host_id', how='left')

# Rimuovi host con prima recensione nel 2025
hosts_2025 = listings[listings['first_review_year'] == 2025]['host_id'].unique()
listings = listings[~listings['host_id'].isin(hosts_2025)]

In [ ]:
# Rimozione outlier di prezzo usando 95° percentile
price_threshold = listings['price_clean'].quantile(0.95)
listings = listings[listings['price_clean'] <= price_threshold].copy()

In [ ]:
# Filtro per tipi di stanza rilevanti (esclude hotel room e shared room)
listings = listings[listings['room_type'].isin(['Entire home/apt', 'Private room'])].copy()

In [ ]:
# Creazione gruppi per numero di persone ospitabili
listings['accommodates_group'] = np.where(listings['accommodates'] >= 8, '8 o più',
                                         listings['accommodates'].astype(str))

print(f"Dataset prima pulizia:")
print(f"- Listings: {len(listings_df)} annunci")
print(f"- Host unici: {listings_df['host_id'].nunique()}")
print(f"- Tipi di stanza: {listings_df['room_type'].unique()}")

print(f"\nDataset dopo pulizia:")
print(f"- Listings: {len(listings)} annunci")
print(f"- Host unici: {listings['host_id'].nunique()}")
print(f"- Limite prezzo (95° percentile): ${price_threshold:.2f}")
print(f"- Tipi di stanza: {listings['room_type'].unique()}")

## Panoramice Dataset

In [ ]:
# Statistiche generali
print(f"Totale annunci: {len(listings)}")
print(f"Annunci unici: {listings['id'].nunique()}")
print(f"Host unici: {listings['host_id'].nunique()}")
print(f"Tipi di stanza disponibili: {listings['room_type'].unique()}")
print(f"Media dei prezzi: ${listings['price_clean'].mean():.2f}")
print(f"Mediana dei prezzi: ${listings['price_clean'].median():.2f}")

sns.histplot(data=listings, x='price_clean', bins=50, kde=True)
plt.show()

In [ ]:
# Divisione per tipo di host
superhost = listings[listings['host_is_superhost'] == 't'].copy()
non_superhost = listings[listings['host_is_superhost'] == 'f'].copy()

print(f"\nDistribuzione per tipo di host:")
print(f"- Superhost: {len(superhost)} ({len(superhost)/len(listings)*100:.1f}%)")
print(f"- Host regolari: {len(non_superhost)} ({len(non_superhost)/len(listings)*100:.1f}%)")

sns.countplot(data=listings, x='host_is_superhost', stat='percent')
plt.show()

In [ ]:
# Conteggio per tipo di stanza
print(f"\nDistribuzione tipi di stanza:")
print(listings['room_type'].value_counts())

sns.countplot(data=listings, x='room_type', stat='percent')
plt.show()

## Statistiche descrittive

In [ ]:
# Statistiche prezzi
print("\nPREZZI ($) - SUPERHOST:")
superhost_price_stats = superhost['price_clean'].describe()
for stat, value in superhost_price_stats.items():
    if stat == 'count':
        print(f"  {stat.capitalize():<6}: {value:.0f}")
    else:
        print(f"  {stat.capitalize():<6}: ${value:.2f}")

print("\nPREZZI ($) - HOST REGOLARI:")
host_price_stats = non_superhost['price_clean'].describe()
for stat, value in host_price_stats.items():
    if stat == 'count':
        print(f"  {stat.capitalize():<6}: {value:.0f}")
    else:
        print(f"  {stat.capitalize():<6}: ${value:.2f}")

In [ ]:
# Statistiche review scores (rimuovendo valori null)
superhost_reviews = superhost['review_scores_rating'].dropna()
host_reviews = non_superhost['review_scores_rating'].dropna()

print("\nREVIEW SCORES - SUPERHOST:")
for stat, value in superhost_reviews.describe().items():
    print(f"  {stat.capitalize():<6}: {value:.2f}")

print("\nREVIEW SCORES - HOST REGOLARI:")
for stat, value in host_reviews.describe().items():
    print(f"  {stat.capitalize():<6}: {value:.2f}")

In [ ]:
# Statistiche numero recensioni
print("\nNUMERO RECENSIONI - SUPERHOST:")
superhost_count_stats = superhost['review_count'].describe()
for stat, value in superhost_count_stats.items():
    print(f"  {stat.capitalize():<6}: {value:.2f}")

print("\nNUMERO RECENSIONI - HOST REGOLARI:")
host_count_stats = non_superhost['review_count'].describe()
for stat, value in host_count_stats.items():
    print(f"  {stat.capitalize():<6}: {value:.2f}")

## Grafici

In [ ]:
# 1. Prezzo medio per tipo di stanza
plt.figure(figsize=(10, 6))
sns.barplot(data=listings, x='room_type', y='price_clean')
plt.title('Prezzo medio per tipo di stanza')
plt.xlabel('Tipo di Stanza')
plt.ylabel('Prezzo Medio ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 2. Distribuzione numero persone ospitabili per tipo stanza
selected_rooms = listings[listings['room_type'].isin(['Private room', 'Entire home/apt'])].copy()

plt.figure(figsize=(12, 6))
sns.countplot(
    data=selected_rooms,
    x='accommodates',
    hue='room_type',
    order=sorted(selected_rooms['accommodates'].unique())
)
plt.title('Distribuzione del numero di persone ospitabili per tipo di stanza')
plt.xlabel('Numero di persone ospitabili')
plt.ylabel('Numero di annunci')
plt.legend(title='Tipo di stanza')
plt.tight_layout()
plt.show()

In [ ]:
# 3. Analisi prezzi per gruppo di persone e tipo di host
grouped_host = listings.groupby(['room_type', 'accommodates_group', 'host_is_superhost']).agg(
    mean_price=('price_clean', 'mean')
).reset_index()

# Conversione per calcoli numerici
grouped_host['accommodates_numeric'] = grouped_host['accommodates_group'].replace({'8 o più': 8}).astype(int)
grouped_host['price_per_person'] = grouped_host['mean_price'] / grouped_host['accommodates_numeric']

# Etichette leggibili
grouped_host['Tipo host'] = grouped_host['host_is_superhost'].map({'t': 'Superhost', 'f': 'Host'})
grouped_host = grouped_host.rename(columns={
    'room_type': 'Tipo stanza',
    'accommodates_group': 'Numero persone (gruppo)',
    'mean_price': 'Prezzo medio ($)',
    'price_per_person': 'Prezzo per persona ($)'
})

# Arrotondamento valori
grouped_host[['Prezzo medio ($)', 'Prezzo per persona ($)']] = grouped_host[[
    'Prezzo medio ($)', 'Prezzo per persona ($)'
]].round(2)

# Stampa tabella riepilogativa
print("\nTabella con confronto tra Host e Superhost:")
print(grouped_host[['Tipo stanza', 'Numero persone (gruppo)', 'Tipo host',
                   'Prezzo medio ($)', 'Prezzo per persona ($)']].to_string(index=False))

In [ ]:
# 4. Grafici comparativi prezzo medio
g1 = sns.FacetGrid(
    grouped_host, col='Tipo stanza', hue='Tipo host',
    height=5, aspect=1.3, margin_titles=True
)
g1.map(sns.lineplot, 'Numero persone (gruppo)', 'Prezzo medio ($)', marker='o')
g1.add_legend(title='Tipo host')
g1.set_axis_labels("Numero persone (gruppo)", "Prezzo medio ($)")
g1.fig.subplots_adjust(top=0.85)
g1.fig.suptitle("Prezzo medio per gruppo - Host vs Superhost", fontsize=16)
plt.show()

In [ ]:
# 5. Grafici comparativi prezzo per persona
g2 = sns.FacetGrid(
    grouped_host, col='Tipo stanza', hue='Tipo host',
    height=5, aspect=1.3, margin_titles=True
)
g2.map(sns.lineplot, 'Numero persone (gruppo)', 'Prezzo per persona ($)', marker='o')
g2.add_legend(title='Tipo host')
g2.set_axis_labels("Numero persone (gruppo)", "Prezzo per persona ($)")
g2.fig.subplots_adjust(top=0.85)
g2.fig.suptitle("Prezzo medio per persona - Host vs Superhost", fontsize=16)
plt.show()

In [ ]:
# 6. Diagramma Sankey per visualizzare flussi
sankey_df = listings.copy()
sankey_df['host_type'] = sankey_df['host_is_superhost'].map({'t': 'Superhost', 'f': 'Host'})

# Preparazione dati per Sankey
cols = ['host_type', 'room_type', 'accommodates_group']
flow_counts = sankey_df.groupby(cols).size().reset_index(name='count')

# Creazione etichette e mapping
labels = pd.unique(flow_counts[cols[0]].tolist() +
                  flow_counts[cols[1]].tolist() +
                  flow_counts[cols[2]].tolist())
label_dict = {label: i for i, label in enumerate(labels)}

# Creazione collegamenti per il diagramma
source, target, value = [], [], []

# Collegamenti host_type → room_type
level1 = flow_counts.groupby([cols[0], cols[1]])['count'].sum().reset_index()
for _, row in level1.iterrows():
    source.append(label_dict[row[cols[0]]])
    target.append(label_dict[row[cols[1]]])
    value.append(row['count'])

# Collegamenti room_type → accommodates_group
level2 = flow_counts.groupby([cols[1], cols[2]])['count'].sum().reset_index()
for _, row in level2.iterrows():
    source.append(label_dict[row[cols[1]]])
    target.append(label_dict[row[cols[2]]])
    value.append(row['count'])

# Assegnazione colori ai collegamenti
color_palette = pc.qualitative.Set3
link_colors = [color_palette[i % len(color_palette)] for i in range(len(source))]

# Creazione e visualizzazione diagramma Sankey
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15, thickness=20,
        line=dict(color="black", width=0.5),
        label=labels, color="lightgray"
    ),
    link=dict(
        source=source, target=target, value=value,
        color=link_colors
    )
)])

fig.update_layout(title_text="Flusso: Host → Tipo stanza → Numero persone", font_size=12)
fig.show()

In [ ]:
# 7. Confronto tipi di alloggi offerti da host e superhost
plt.figure(figsize=(10, 6))
sns.countplot(data=listings, x='host_is_superhost', hue='room_type')
plt.title('Tipi di alloggi che offrono host e super host')
plt.xlabel('Tipo di Host (f=Host, t=Superhost)')
plt.ylabel('Numero di alloggi')
plt.show()

In [ ]:
# 8. Confronto tempi di risposta
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Superhost tempi di risposta
superhost_response = superhost['host_response_time'].value_counts(normalize=True)
sns.barplot(x=superhost_response.index, y=superhost_response.values, ax=axes[0])
axes[0].set_title('Tempi di risposta - Superhost')
axes[0].set_xlabel('Tempo di risposta')
axes[0].set_ylabel('Percentuale')
axes[0].tick_params(axis='x', rotation=45)

# Host regolari tempi di risposta
host_response = non_superhost['host_response_time'].value_counts(normalize=True)
sns.barplot(x=host_response.index, y=host_response.values, ax=axes[1])
axes[1].set_title('Tempi di risposta - Host regolari')
axes[1].set_xlabel('Tempo di risposta')
axes[1].set_ylabel('Percentuale')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Analisi Amenities

In [ ]:
# Parsing amenities per superhost
all_amenities_super = []
for item in superhost['amenities'].dropna():
    item = item.strip('{}, []').replace('"', '').split(',')
    all_amenities_super.extend([x.strip() for x in item])
superhost_amenities = Counter(all_amenities_super)

# Parsing amenities per host regolari
all_amenities_host = []
for item in non_superhost['amenities'].dropna():
    item = item.strip('{}').replace('"', '').split(',')
    all_amenities_host.extend([x.strip() for x in item])
host_amenities = Counter(all_amenities_host)

In [ ]:
# Calcolo percentuali top 20 amenities
top_amenities_super = pd.Series({k: v/len(superhost) for k, v in superhost_amenities.items()}).sort_values(ascending=False).head(20)
top_amenities_host = pd.Series({k: v/len(non_superhost) for k, v in host_amenities.items()}).sort_values(ascending=False).head(20)

print("\nTop 20 Amenities Superhost (percentuale):")
for amenities, pct in top_amenities_super.items():
    print(f"  {amenities}: {pct:.1%}")

print("\nTop 20 Amenities Host regolari (percentuale):")
for amenities, pct in top_amenities_host.items():
    print(f"  {amenities}: {pct:.1%}")

In [ ]:
# Creazione dataframe per confronto grafico
df_amenities = pd.DataFrame({
    'Superhost': top_amenities_super,
    'Host regolari': top_amenities_host
}).fillna(0)

# Grafico comparativo amenities
plt.figure(figsize=(12, 6))
df_amenities.plot(kind='bar', figsize=(12, 6))
plt.title("Percentuale Top 20 Amenities - Superhost vs Host regolari")
plt.xlabel("Percentuale di annunci")
plt.ylabel("Amenities")
plt.xticks(rotation=45, ha='right')
plt.legend(title='Tipo Host')
plt.tight_layout()
plt.show()

In [ ]:
# Aggrega il numero di recensioni per host
host_reviews = listings.groupby('host_id')['review_count'].sum().reset_index()

# Unisci con info host (amenities + superhost)
host_data = listings[['host_id', 'amenities', 'host_is_superhost']].drop_duplicates()
host_info = pd.merge(host_reviews, host_data, on='host_id', how='left')

# Parsing semplice delle amenities
host_info['parsed_amenities'] = host_info['amenities'].fillna('').apply(
    lambda x: [a.strip() for a in x.strip('{},[]').replace('"', '').split(',') if a.strip()]
)

# Esplosione su ogni amenities
host_amenities = host_info.explode('parsed_amenities')
host_amenities = host_amenities[host_amenities['parsed_amenities'] != '']

# Aggregazione: somma recensioni per amenities + tipo host
agg = host_amenities.groupby(['parsed_amenities', 'host_is_superhost'])['review_count'].sum().reset_index()
agg.columns = ['amenities', 'Tipo Host', 'Totale Recensioni']

# Mappatura tipo host leggibile
agg['Tipo Host'] = agg['Tipo Host'].map({'t': 'Superhost', 'f': 'Host Regolare'})

# Top amenities per Superhost
top_n = 20
top_amenities = (
    agg[agg['Tipo Host'] == 'Superhost']
    .sort_values('Totale Recensioni', ascending=False)
    .head(top_n)['amenities']
    .tolist()
)


# Filtro solo sulle top amenities per il confronto
filtered = agg[agg['amenities'].isin(top_amenities)].copy()

# Ordina le amenities per Totale Recensioni dei Superhost
order = (
    filtered[filtered['Tipo Host'] == 'Superhost']
    .sort_values('Totale Recensioni', ascending=False)['amenities']
    .tolist()
)

# Imposta l'ordinamento nelle categorie per l’asse Y
filtered['amenities'] = pd.Categorical(filtered['amenities'], categories=order, ordered=True)

# Plot con Seaborn
plt.figure(figsize=(12, 6))
sns.barplot(
    data=filtered,
    x='Totale Recensioni',
    y='amenities',
    hue='Tipo Host',
    palette='magma'
)

plt.title(f"Totale Recensioni per Top {top_n} Amenities - Superhost vs Host Regolare", fontsize=14)
plt.xlabel("Totale Recensioni")
plt.ylabel("amenities")
plt.legend(title="Tipo Host")
plt.tight_layout()
plt.show()

## Analisi distribuzione recensioni

In [ ]:
# Statistiche recensioni superhost
super_with_reviews = (superhost['review_count'] > 0).mean()
super_many_reviews = (superhost['review_count'] > 10).mean()

print(f"\nSuperhost:")
print(f"  Annunci con almeno 1 recensione: {super_with_reviews:.2%}")
print(f"  Annunci con più di 10 recensioni: {super_many_reviews:.2%}")

# Statistiche recensioni host regolari
host_with_reviews = (non_superhost['review_count'] > 0).mean()
host_many_reviews = (non_superhost['review_count'] > 10).mean()

print(f"\nHost regolari:")
print(f"  Annunci con almeno 1 recensione: {host_with_reviews:.2%}")
print(f"  Annunci con più di 10 recensioni: {host_many_reviews:.2%}")

In [ ]:
!pip install folium geopandas

import folium
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement

# Calcolo totale recensioni per ogni location
location_reviews = listings.groupby(
    ['id', 'name', 'latitude', 'longitude', 'host_is_superhost', 'price_clean']
)['review_count'].sum().reset_index()
location_reviews.rename(columns={'review_count': 'total_reviews'}, inplace=True)

# Prendi le top 100 location per numero di recensioni
top_100_locations = location_reviews.sort_values(by='total_reviews', ascending=False).head(100).copy()

# Crea mappa centrata sulla Sicilia
sicily_center = [37.5, 14.0]
m = folium.Map(location=sicily_center, zoom_start=8)

# Aggiungi MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

# Colori per host e superhost
host_color = 'blue'
superhost_color = 'red'

# Aggiungi marker per ogni location
for _, row in top_100_locations.iterrows():
    host_type = 'Superhost' if row['host_is_superhost'] == 't' else 'Host'
    marker_color = superhost_color if row['host_is_superhost'] == 't' else host_color
    popup_text = f"""
    <b>{row['name']}</b><br>
    Tipo Host: {host_type}<br>
    Recensioni totali: {int(row['total_reviews'])}<br>
    Prezzo: ${row['price_clean']:.2f}
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color=marker_color)
    ).add_to(marker_cluster)

# Aggiungi legenda semplice
template = f"""
{{% macro html(this, kwargs) %}}
<div style="position: fixed; bottom: 50px; left: 50px; width: 150px; z-index:9999; font-size:14px;">
  &nbsp; Legenda <br>
  &nbsp; <i class="fa fa-map-marker fa-2x" style="color:{superhost_color}"></i>&nbsp; Superhost <br>
  &nbsp; <i class="fa fa-map-marker fa-2x" style="color:{host_color}"></i>&nbsp; Host <br>
</div>
{{% endmacro %}}
"""
macro = MacroElement()
macro._template = Template(template)
m.get_root().add_child(macro)

# Visualizza la mappa
m


In [ ]:
from folium.plugins import HeatMap

# Calcolo totale recensioni per ogni location
location_reviews = listings.groupby(
    ['id', 'name', 'latitude', 'longitude', 'host_is_superhost', 'price_clean']
)['review_count'].sum().reset_index()
location_reviews.rename(columns={'review_count': 'total_reviews'}, inplace=True)

# Prendi le location per numero di recensioni
top_100_locations = location_reviews.sort_values(by='total_reviews', ascending=False).copy()

# Crea mappa centrata sulla Sicilia
sicily_center = [37.5, 14.0]
m = folium.Map(location=sicily_center, zoom_start=8)

# Prepara i dati per la heatmap: [lat, lon, peso]
heat_data = [
    [row['latitude'], row['longitude'], row['total_reviews']]
    for _, row in top_100_locations.iterrows()
]

# Aggiungi HeatMap
HeatMap(heat_data, radius=15, max_zoom=12).add_to(m)

# Visualizza la mappa
m


In [ ]:
# Filtra il dataframe per includere solo i superhost
superhost_locations = listings[listings['host_is_superhost'] == 't'].copy()

# Calcola il totale recensioni per ogni location dei superhost
location_reviews_superhost = superhost_locations.groupby(
    ['id', 'name', 'latitude', 'longitude', 'price_clean']
)['review_count'].sum().reset_index()
location_reviews_superhost.rename(columns={'review_count': 'total_reviews'}, inplace=True)

# Prendi le location per numero di recensioni (non solo top 100, prendiamo tutti i superhost)
sorted_superhost_locations = location_reviews_superhost.sort_values(by='total_reviews', ascending=False).copy()

# Crea mappa centrata sulla Sicilia
sicily_center = [37.5, 14.0]
m_superhost_heatmap = folium.Map(location=sicily_center, zoom_start=8)

# Prepara i dati per la heatmap: [lat, lon, peso]
# Usa il numero di recensioni come peso per la heatmap
heat_data_superhost = [
    [row['latitude'], row['longitude'], row['total_reviews']]
    for _, row in sorted_superhost_locations.iterrows()
]

# Aggiungi HeatMap solo per i superhost
HeatMap(heat_data_superhost, radius=15, max_zoom=12).add_to(m_superhost_heatmap)

# Visualizza la mappa dei superhost
print("Heatmap solo per Superhost:")
m_superhost_heatmap